In [1]:
### 0. 임포트
# API 호출 관련
import bs4
import json
import requests
import psycopg2
from os import name
import numpy  as np
import pandas as pd
from tqdm import tqdm

# 날짜 관련
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

In [2]:
### 1. 시군구코드 API
lawd_cd_df = pd.DataFrame()
lawd_cd_list = []
lawd_cd_unique = []

for i in range(1, 22):
    
    url = 'http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList'
    serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
    params = {
        'serviceKey' : serviceKey,
        'pageNo' : str(i),
        'numOfRows' : '1000',
        'type' : 'json'
    }
    response = requests.get(url, params=params)
    response.encoding = 'utf-8'
    content = response.text
    
    # Json List 변환
    lawd_cd_df = pd.DataFrame(pd.json_normalize(data = json.loads(content), record_path = 'StanReginCd')['row'][1])
    lawd_cd_list = list(lawd_cd_df['locatjumin_cd'].str[:2]) # 시도 단위
    
    # 시군구코드 unique하게 남기기
    for x in lawd_cd_list:
        if x not in lawd_cd_unique:
            lawd_cd_unique.append(x)

In [10]:
lawd_cd_unique

['27',
 '30',
 '26',
 '28',
 '29',
 '41',
 '31',
 '36',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '11',
 '50']

In [20]:
### 2. 토지실거래가 API 호출
url = 'http://apis.data.go.kr/1613000/BusSttnInfoInqireService/getSttnNoList'
serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='

# 리스트 생성
row_list   = [] # 행 값
name_list  = [] # 열이름 값
value_list = [] # 데이터 값
DF = pd.DataFrame() # 최종 값

In [ ]:
# 26, 25, 

In [65]:
params ={'serviceKey' : serviceKey, 'numOfRows' : '100000', '_type' : 'xml', 'cityCode' : '31'}
response = requests.get(url, params=params)
content  = response.text
content

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items/><numOfRows>100000</numOfRows><pageNo>1</pageNo><totalCount>0</totalCount></body></response>'

In [53]:
# item 태그 분리
xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows    = xml_obj.findAll('item')

# 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()

    for j in range(0,len(columns)):
        name_list.append(columns[j].name)
        value_list.append(columns[j].text)

    row_list = value_list
    df_each  = pd.DataFrame([row_list], columns=name_list)
    DF = pd.concat([DF, df_each]).reset_index(drop = True)

    # 초기화
    name_list  = []
    value_list = []

In [54]:
DF

,gpslati,gpslong,nodeid,nodenm,nodeno
0,36.410934,127.337074,DJB8001191,군인아파트,43730
1,36.36067,127.34951,DJB8001192,궁동,42690
2,36.360806,127.34954,DJB8001193,궁동,42700
3,36.363605,127.3376,DJB8001194,장대네거리,42120
4,36.364986,127.33864,DJB8001195,장대네거리,42140
...,...,...,...,...,...
6061,36.255295,127.34007,DJB8003037,흑석리역,33690
6062,36.255207,127.33997,DJB8003038,흑석리역,33680
6063,36.39692,127.37347,DJB8003041,국가정보자원관리원,43500
6064,36.33853,127.376175,DJB8003043,KT인재개발원,31350


In [ ]:
# API 요청
for code in tqdm(lawd_cd_unique):
    params = {
          'serviceKey' : serviceKey
        , 'cityCode'  : code  # 시도 단위
    }
    response = requests.get(url, params=params)
    content  = response.text

    # item 태그 분리
    xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
    rows    = xml_obj.findAll('item')

    # 데이터 수집
    for i in range(0, len(rows)):
        columns = rows[i].find_all()

        for j in range(0,len(columns)):
            name_list.append(columns[j].name)
            value_list.append(columns[j].text)

        row_list = value_list
        df_each  = pd.DataFrame([row_list], columns=name_list)
        DF = pd.concat([DF, df_each]).reset_index(drop = True)

        # 초기화
        name_list  = []
        value_list = []

# 전처리
DF = DF.replace(' ', None)
DF = DF.replace(np.nan, None)
if '구분' in DF.columns:
    DF = DF.drop(['구분'], axis = 1)

# rename
DF = DF.rename(columns = {"거래금액":"deal_amount",
                          "거래면적":"building_area",
                          "거래유형":"trade_type",
                          "년":"deal_year",
                          "법정동":"dong",
                          "시군구":"sigungu",
                          "용도지역":"land_use",
                          "월":"deal_month",
                          "일":"deal_day",
                          "중개사소재지":"broker_location",
                          "지목":"land_category",
                          "지역코드":"regional_code",
                          "해제사유발생일":"unregister_day",
                          "해제여부":"unregister_status"
                          })

# dtype 변경
DF['deal_amount'] = DF['deal_amount'].apply(lambda x: x.replace(',', '')).astype('int64')

# 날짜 두 자리수로 맞추기
DF['deal_day'] = ('0' + DF['deal_day'].astype(str)).str[-2:]
DF['deal_month'] = ('0' + DF['deal_month'].astype(str)).str[-2:]

In [ ]:
### 3. DB INSERT
# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(DF.columns))[:-1] + ')'

# insert
for j in range(0, len(DF), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = DF[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(DF))